<a href="https://colab.research.google.com/github/Robert0912/Solucion_Sudokus_con_ANN/blob/main/Sudokus_Version2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preprocesamiento de datos

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/drive/MyDrive/Documents/Universidad/sudoku2.csv')

feat_raw = data['quizzes']
label_raw = data['solutions']

feat = []
label = []

for i in feat_raw:

    x = np.array([int(j) for j in i]).reshape((9,9,1))
    feat.append(x)

feat = np.array(feat)
feat = feat/9
feat -= .5    

for i in label_raw:

    x = np.array([int(j) for j in i]).reshape((81,1)) - 1
    label.append(x)   

label = np.array(label)

del(feat_raw)
del(label_raw)    

X_train, X_test, Y_train, Y_test = train_test_split(feat, label, test_size=0.2)

##Entrenamiento del Modelo

In [5]:
import keras
from keras.layers import Activation
from keras.layers import BatchNormalization, Dense, Flatten, Reshape

model = keras.models.Sequential()

model.add(Dense(64, activation='relu', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

model.fit(X_train, Y_train, batch_size=32, epochs=2)
model.save("modelo2.h5")

Epoch 1/2
25000/25000 [==============================] - 2143s 86ms/step - loss: 0.4379
Epoch 2/2
25000/25000 [==============================] - 2117s 85ms/step - loss: 0.3914


##Funciones para la solucionar modelos 

In [6]:
def norm(a):
    
  return (a/9)-.5

def denorm(a):
    
  return (a+.5)*9

import copy

def inference_sudoku(sample):
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

def rendimiento(x,y,y_predict):

  diff = y_predict [[0]] - y[[0]]

  iguales_y_vs_ypredict=(float(np.count_nonzero(diff==0)))

  correctos_previos=np.count_nonzero(x[[0]])

  diferencia=iguales_y_vs_ypredict-correctos_previos

  if diferencia>0:

    return diferencia/((81)-correctos_previos)

  else:

    return diferencia/(correctos_previos)

##Mediciones de exactitud

In [8]:
test_accuracy(X_test[:100], Y_test[:100])

0.91


##Soluciones de Sudokus al azar

In [9]:
def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [10]:
game1 = '''
          0 6 0 1 0 4 0 5 0
          0 0 8 3 0 5 6 0 0
          2 0 0 0 0 0 0 0 1
          8 0 0 4 0 7 0 0 6
          0 0 6 0 0 0 3 0 0
          7 0 0 9 0 1 0 0 4
          5 0 0 0 0 0 0 0 2
          0 0 7 2 0 6 9 0 0
          0 4 0 5 0 8 0 7 0
      '''

game1 = solve_sudoku(game1)

print('Sudoku Resuelto:\n')
print(game1)

Sudoku Resuelto:

[[3 6 9 1 2 4 7 5 8]
 [4 1 8 3 7 5 6 2 9]
 [2 7 5 6 8 9 4 3 1]
 [8 5 1 4 3 7 2 9 6]
 [9 4 6 8 5 2 3 1 7]
 [7 2 3 9 6 1 5 8 4]
 [5 9 4 7 1 3 8 6 2]
 [1 3 7 2 4 6 9 4 5]
 [6 4 2 5 9 8 1 7 3]]


In [11]:
game2 = '''
          0 8 0 0 3 2 0 0 1
          7 0 3 0 8 0 0 0 2
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''

game2 = solve_sudoku(game2)

print('Sudoku Resuelto:\n')
print(game2)

Sudoku Resuelto:

[[4 8 9 5 3 2 7 6 1]
 [7 1 3 4 8 6 5 9 2]
 [5 6 2 9 1 7 8 3 4]
 [2 5 8 3 4 1 9 7 6]
 [6 3 1 7 5 9 2 4 8]
 [9 4 7 2 6 8 1 5 3]
 [1 2 5 6 7 3 4 8 9]
 [8 7 6 1 9 4 3 2 5]
 [3 9 4 8 2 5 6 1 7]]


In [12]:
game3 = '''
          5 3 0 0 7 0 0 0 0 
          6 0 0 1 9 5 0 0 0
          0 9 8 0 0 0 0 6 0
          8 0 0 0 6 0 0 0 3
          4 0 0 8 0 3 0 0 1
          7 0 0 0 2 0 0 0 6
          0 6 0 0 0 0 2 8 0
          0 0 0 4 1 9 0 0 5
          0 0 0 0 8 0 0 7 9
      '''

game3 = solve_sudoku(game3)

print('Sudoku Resuelto:\n')
print(game3)

Sudoku Resuelto:

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]
